In [ ]:
!pip3 install vk_api

In [ ]:
import vk_api
import datetime
import time
import pandas as pd

In [ ]:
def two_factor_code():
     return input('Введите код двухфакторной аутентификации: '), True

In [ ]:
def auth():
    # Укажите логин и пароль от вашего аккаунта VK
    login = 'login'
    password = 'password'

    # Авторизуемся в VK API
    vk_session = vk_api.VkApi(login, password)


    # Проверяем, требуется ли двухфакторная аутентификация
    try:
        vk_session.auth()
    except vk_api.AuthError as error_msg:
        
        # Если требуется двухфакторная аутентификация, то запросим код и повторим попытку авторизации
        vk_session = vk_api.VkApi(login, password, auth_handler=two_factor_code)
        vk_session.auth()

    # Получаем объект для работы с методами API VK
    vk = vk_session.get_api()
    tools = vk_api.VkTools(vk_session)
    return vk, tools

In [ ]:
vk, tools = auth()

In [ ]:
df_posts = pd.DataFrame()
df_comments = pd.DataFrame()

# Укажите ID или короткое имя сообщества, со стены которого нужно получить сообщения
group_id = -1437634
offset = 0


while True:
    all_posts = vk.wall.get(owner_id=group_id, offset=offset, count=100)['items']
    
    # Прерываем цикл, если приходит пустой ответ
    if not all_posts:
        break

    # Сохраняем сообщения и комментарии к ним
    for post in all_posts:
        post['date'] = datetime.datetime.fromtimestamp(post['date'])
        post_id = post['id']
        df_posts = pd.concat([df_posts, pd.json_normalize(post)])
        comments = vk.wall.getComments(owner_id=group_id, post_id=post_id)
        for comment in comments['items']:
            comment['date'] = datetime.datetime.fromtimestamp(comment['date'])
            df_comments = pd.concat([df_comments, pd.json_normalize(comment)])
    
    # Смещение на число уже обработанных сообщений
    offset += len(all_posts)
    
    # Контролируем ход работы кода
    min_date = min(df_posts.date)
    max_date = max(df_posts.date)
    print(offset)
    print(min_date, max_date)
    
    time.sleep(0.1)
    
df_posts.head()

In [ ]:
df_comments.to_csv('comments.csv')
df_posts.to_csv('posts.csv')